In [ ]:
# Installation and Importation of machine learning libraries
!pip install scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

print("Machine Learning tools ready!")

You should consider upgrading via the 'C:\Users\hp\Desktop\Beliro\PUIT22217146\Beliro_env\Scripts\python.exe -m pip install --upgrade pip' command.


Machine Learning tools ready!


In [6]:
# Filter and Split (UPDATED)
df = pd.read_csv('02_cleaned_data.csv')

# --- THE FIX: REMOVE OUTLIERS ---
# Only keep houses priced between $10 and $500
# This removes "glitch" $0 listings and the confusing $10,000 mansions
df = df[(df['price'] > 10) & (df['price'] < 500)]

print(f"Data filtered. Remaining rows: {len(df)}")

# Re-define X and y with this cleaner data
X = df.drop(['price', 'price_log'], axis=1)
y = df['price_log'] # We still use log price for training

# Split again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Data filtered. Remaining rows: 47595


In [9]:
# Training with XGBoost
from xgboost import XGBRegressor

print("Igniting the XGBoost engine...")

# 1. Setup
# n_estimators=1000: Give it 1000 tries to fix its errors
# learning_rate=0.05: Learn slowly and carefully (prevent rushing to wrong conclusions)
# max_depth=7: Allow it to make deeper, more complex connections
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=7, n_jobs=-1, random_state=42)

# 2. Train
model.fit(X_train, y_train)
print("Training complete.")

Igniting the XGBoost engine...
Training complete.


In [10]:
# Model Evaluation (Revisited)
predictions_log = model.predict(X_test)

# Convert log prices back to real dollars
predictions_real = np.expm1(predictions_log)
y_test_real = np.expm1(y_test)

# Calculate scores
mae = mean_absolute_error(y_test_real, predictions_real)
accuracy = r2_score(y_test_real, predictions_real)

print(f"New Average Error: ${mae:.2f}")
print(f"New Accuracy Score (R2): {accuracy:.2f}")

if accuracy > 0.6:
    print("🚀 BOOM! We crossed the 0.60 barrier. That is impressive for data without square footage!")
else:
    print("We improved, but the missing 'Size' data is limiting us.")

New Average Error: $36.78
New Accuracy Score (R2): 0.54
We improved, but the missing 'Size' data is limiting us.
